# Importación de librerias

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from scipy.spatial import distance

# Metodo de distancias

In [2]:
def euclideanDistance(a, b):
    return distance.euclidean(a.values, b.values)
    #return np.linalg.norm(a.values - b.values)

def manhattanDistance(a, b):
    return distance.cityblock(a.values, b.values)

def cosDistance(a,b):
    return distance.cosine(a.values, b.values)

#Distancias a calcular
distancesMethods = [1, 2, 3]

# Lectura de archivo y eliminación de registros no válidos
Se eliminan los registros que NO hayan escrito la palabra greyc laboratory

In [3]:
path = "./data/grey/keystroke.db" 
conn = sqlite3.connect(path)
df = pd.read_sql_query('select * from keystroke_datas', conn, parse_dates=['date'])
df.drop(df[df['password'] != 'greyc laboratory'].index, inplace = True)
conn.close()

# Previsualización del dataset original

In [4]:
df.head(5)

,id,ppTime,rrTime,prTime,rpTime,vector,password,user_id,date,time_to_type,rawPress,rawRelease
5,6,2203168 600864 1101584 1602304 801152 2303312...,2203168 901296 500720 2503600 600864 1902736 ...,3204608 1902736 1802592 3204608 2203168 33047...,1201728 -400576 -200288 901296 -801152 901296...,2203168 600864 1101584 1602304 801152 2303312...,greyc laboratory,1,2009-03-18 08:49:09,30644064,71 633729665463844160\n82 633729665466047328\n...,71 633729665464845600\n82 633729665467048768\n...
6,7,2103024 500720 2703888 1602304 2103024 340489...,1902736 701008 2203168 1802592 2203168 310446...,3104464 1702448 3404896 2503600 3104464 41059...,901296 -500720 1502160 901296 1201728 2403456...,2103024 500720 2703888 1602304 2103024 340489...,greyc laboratory,1,2009-03-18 08:49:18,33748528,71 633729665547564544\n82 633729665549667568\n...,71 633729665548766272\n82 633729665550669008\n...
7,8,2203168 701008 1402016 1301872 2303312 330475...,2503600 801152 901296 1602304 2203168 3104464...,3304752 1902736 2103024 2303312 3204608 40057...,1402016 -400576 200288 600864 1301872 2403456...,2203168 701008 1402016 1301872 2303312 330475...,greyc laboratory,1,2009-03-18 08:49:28,29442336,71 633729665651914592\n82 633729665654117760\n...,71 633729665652715744\n82 633729665655219344\n...
8,9,2103024 701008 1902736 1702448 1602304 260374...,2303312 701008 1402016 2002880 1602304 240345...,3304752 1902736 2603744 2703888 2603744 34048...,1101584 -500720 701008 1001440 600864 1602304...,2103024 701008 1902736 1702448 1602304 260374...,greyc laboratory,1,2009-03-18 08:49:48,31545360,71 633729665853404320\n82 633729665855507344\n...,71 633729665854405760\n82 633729665856709072\n...
9,10,2002880 600864 1201728 1602304 1502160 300432...,2002880 901296 701008 1802592 1502160 2703888...,2904176 1802592 1902736 2503600 2403456 36051...,1101584 -300432 0 901296 600864 2103024 50072...,2002880 600864 1201728 1602304 1502160 300432...,greyc laboratory,1,2009-03-18 08:49:58,29642624,71 633729665954750048\n82 633729665956752928\n...,71 633729665955651344\n82 633729665957654224\n...


# Preprocesamiento de vectores de tiempo

In [5]:
tempData = []

n_data_rows = df.shape[0]
        

columns = ["user_id"]   

for i in range(60):
    columns.append("ft_" + str(i+1))
        
for i in range(n_data_rows):
    raw_id = [df.iloc[i]["id"]]
    user_id = [df.iloc[i]["user_id"]]
    time_to_type = [df.iloc[i]["time_to_type"]]
    
    vector = df.iloc[i]["vector"].split()      
 
    
    if(len(vector) == 60 ):
        tempData.append(user_id  + list(map(int, vector)))
                                                
df = pd.DataFrame(tempData, columns = columns  )

# Previsualización de dataset procesado

In [6]:
df.head(5)

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
0,1,2203168,600864,1101584,1602304,801152,2303312,1001440,2603744,1402016,...,3304752,2002880,3605184,2002880,2804032,2403456,2904176,2403456,2303312,2103024
1,1,2103024,500720,2703888,1602304,2103024,3404896,1402016,2603744,1402016,...,4105904,2303312,3505040,2103024,2203168,2603744,2804032,2603744,3104464,2103024
2,1,2203168,701008,1402016,1301872,2303312,3304752,901296,3004320,1502160,...,4005760,1902736,3905616,2203168,2303312,3104464,2603744,2203168,2203168,2603744
3,1,2103024,701008,1902736,1702448,1602304,2603744,1502160,3004320,1502160,...,3404896,2703888,4005760,2203168,2303312,2403456,3104464,2103024,2904176,2403456
4,1,2002880,600864,1201728,1602304,1502160,3004320,1101584,2904176,1602304,...,3605184,2203168,3905616,2303312,2603744,3104464,2603744,1902736,2503600,2203168


# Separación de la data de entrenamiento y de prueba
Se toma de forma aleatoria el 80% de los registros de cada usuario para considerarlos como data de entrenamiento, mientras que el restante forma a pasar a la data de prueba

In [7]:
subjects = df["user_id"].unique()

tempData.clear()
train_users = []
test_users = []

#Separar 80 / 20

for subject in subjects:
    genuine_user_data = df.loc[df.user_id == subject, :]
            
    imposter_data = df.loc[df.user_id != subject, :]

    train, test_genuine  = train_test_split(genuine_user_data, train_size = 0.8, random_state=43, shuffle=True)

    train_users.append(train)
    test_users.append(test_genuine)

# Cálculo del vecto con la media de cada usuario

In [8]:
#Se calcula la media de cada usuario
train_users = pd.concat(train_users)
groupby = train_users.groupby("user_id").mean()

train_users = groupby.reset_index()
test_users = pd.concat(test_users)

train_users.head(7)

,user_id,ft_1,ft_2,ft_3,ft_4,ft_5,ft_6,ft_7,ft_8,ft_9,...,ft_51,ft_52,ft_53,ft_54,ft_55,ft_56,ft_57,ft_58,ft_59,ft_60
0,1,1.999346e+06,6.256055e+05,1.459746e+06,1.134573e+06,9.166121e+05,2.333944e+06,9.319283e+05,2.349260e+06,1.564603e+06,...,3.165729e+06,2.006414e+06,3.421390e+06,2.306846e+06,2.921848e+06,2.574290e+06,2.546014e+06,1.840293e+06,2.821704e+06,2.002880e+06
1,2,2.329025e+06,2.212641e+06,1.679442e+06,1.479154e+06,8.674636e+05,2.583445e+06,1.407429e+06,1.825598e+06,2.874403e+06,...,3.353471e+06,2.632163e+06,2.545552e+06,3.682322e+06,2.426462e+06,2.542846e+06,4.132970e+06,3.101757e+06,3.193782e+06,2.496834e+06
2,3,1.666499e+06,2.180058e+06,3.614171e+06,1.782050e+06,2.906744e+06,2.769367e+06,2.086333e+06,3.858112e+06,2.544685e+06,...,3.752832e+06,3.095477e+06,4.827454e+06,3.653972e+06,4.537294e+06,4.032722e+06,3.965959e+06,3.729722e+06,3.412599e+06,4.108472e+06
3,4,1.280789e+06,2.746054e+06,2.322638e+06,1.394988e+06,1.224568e+06,1.698934e+06,2.378859e+06,1.950173e+06,1.955443e+06,...,2.559821e+06,3.092166e+06,2.674020e+06,2.709159e+06,3.290697e+06,2.682805e+06,2.057344e+06,3.592886e+06,3.037701e+06,2.389401e+06
4,5,8.909871e+05,4.447572e+05,1.403489e+06,8.085155e+05,1.818792e+06,2.672961e+06,1.213510e+06,1.556650e+06,2.262076e+06,...,3.248789e+06,1.941026e+06,2.150151e+06,2.854104e+06,2.989593e+06,1.530141e+06,2.615526e+06,1.876227e+06,1.918936e+06,2.169296e+06
5,6,1.112125e+06,6.360022e+05,2.002880e+06,1.625144e+06,1.881653e+06,2.663479e+06,1.191187e+06,2.888364e+06,1.855299e+06,...,3.538421e+06,2.210196e+06,3.754522e+06,2.666993e+06,3.202851e+06,2.053830e+06,3.315293e+06,2.793491e+06,2.746054e+06,2.348992e+06
6,7,1.434899e+06,1.322798e+06,1.564937e+06,2.176264e+06,1.300377e+06,1.318314e+06,1.370628e+06,2.745739e+06,1.512623e+06,...,2.306301e+06,2.088077e+06,3.484114e+06,2.319754e+06,2.925102e+06,2.381036e+06,2.798053e+06,2.493137e+06,4.232952e+06,2.478190e+06


# Cálculo de las distancias y evaluación
1. Se calcula la distancias entre la media de cada usuario (vector) con el vector del test
2. Se escoge la media con la menor distancia encontrada
3. Se chequea si la media corresponde con el usuario del test 
3. Se comienza a contar las veces que se acierta el usuario y las veces que no
5. Se imprime los resultados

In [9]:
for distancesMethod in distancesMethods:
    countGood = 0
    countFalse = 0
    
    for i in range(test_users.shape[0]):
        value = []
        for j in range(train_users.shape[0]):
            if 1 == distancesMethod:
                value.append(euclideanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 2 == distancesMethod:
                value.append(manhattanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 3 == distancesMethod:
                value.append(cosDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
        idx = np.argmin(value)
        
        if train_users.iloc[idx]["user_id"] == test_users.iloc[i]["user_id"] :
            countGood += 1
            #print( "raw "+ str(i)+ " Total: "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
        else:
            countFalse += 1
            #print( "raw "+ str(i)+ " false")
            
    print("----------------------------------------------------")
    if 1 == distancesMethod:
        print("Resumen distancia euclidiana:")
    elif 2 == distancesMethod:
        print("Resumen distancia Manhattan:")
    elif 3 == distancesMethod:
        print("Resumen distancia Coseno:")
            
    print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
    print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0]) +"==> " +str(round(countFalse*100/test_users.shape[0],2)) + "%")

----------------------------------------------------
Resumen distancia euclidiana:
Total acertados 810 / 1556==> 52.06%
Total equivocados 746 / 1556==> 47.94%
----------------------------------------------------
Resumen distancia Manhattan:
Total acertados 960 / 1556==> 61.7%
Total equivocados 596 / 1556==> 38.3%
----------------------------------------------------
Resumen distancia Coseno:
Total acertados 838 / 1556==> 53.86%
Total equivocados 718 / 1556==> 46.14%
